# PI 7 Deel 1 - Mijn Aanpak

In deze opdracht werk ik aan het implementeren en aanpassen van een genetisch algoritme (GA). Ik gebruik de basiscode die is meegeleverd en onderzoek verschillende mogelijkheden voor de belangrijkste stappen van het algoritme: parentselectie, crossover en mutatie. Door relevante literatuur te bestuderen, maak ik onderbouwde keuzes voor de invulling van deze stappen en pas ik de code hierop aan.

## Doel van de opdracht

Het doel is om inzicht te krijgen in de flexibiliteit van genetische algoritmes en te leren hoe verschillende keuzes invloed hebben op de prestaties van het algoritme voor een specifiek probleem. Door te experimenteren met verschillende methoden voor selectie, crossover en mutatie, kan ik bepalen welke aanpak het beste werkt voor deze opdracht.

## Mijn werkwijze

- **Literatuurstudie:** Ik heb verschillende methoden voor selectie, crossover en mutatie onderzocht.
- **Keuze en implementatie:** Op basis van mijn bevindingen heb ik keuzes gemaakt en deze geïmplementeerd in de code.
- **Evaluatie:** Ik heb de prestaties van het algoritme geëvalueerd en mijn keuzes waar nodig aangepast.

In [25]:
import random
import math
import pandas as pd

In [26]:
df = pd.read_csv('pi-7-voorbeeld-dataset.csv')  # Importeren van dataset

# toegevoegd: bewaar originele id (als CSV geen id-kolom heeft, gebruik rijindex)
if 'id' not in df.columns:
    df['job_id'] = df.index  # of use df.index+1 voor 1-based ids
else:
    df['job_id'] = df['id']

print("\nVoor sorteren:")
print(df)

df = df.sort_values(by=['profit'], ascending=False).reset_index(drop=True)
# let op: reset_index verandert rijindices; job_id blijft de originele id
print("\nNa sorteren:")
print(df)


Voor sorteren:
      id  deadline  profit
0     95         6     100
1     23        10     100
2    155         6      99
3     57         2      97
4    162         8      97
..   ...       ...     ...
195   63         8       3
196  118         3       2
197   55         2       1
198  181         3       1
199   15        10       1

[200 rows x 3 columns]

Na sorteren:
      id  deadline  profit
0     95         6     100
1     23        10     100
2    155         6      99
3     57         2      97
4    162         8      97
..   ...       ...     ...
195   63         8       3
196  118         3       2
197   55         2       1
198  181         3       1
199   15        10       1

[200 rows x 3 columns]


### Initialisatie van de populatie

De functie maakt een **initiële populatie**:  
- `initialise_population`: genereert `n_pop` willekeurige permutaties van `0..n_jobs-1`  
- Parameters (`N_POP`, `N_JOBS`, `N_ITERS`, `MUTATION_RATE`) stellen de GA in  
- Daarna wordt de populatie aangemaakt en geprint

In [27]:
def initialise_population(n_pop, n_jobs):
    # Genereer n_pop lijsten, elk een willekeurige volgorde van 0..n_jobs-1
    return [random.sample(range(n_jobs), n_jobs) for i in range(n_pop)]

def fitness(solution, df):
    n = len(df)
    deadlines = df['deadline'].tolist()
    profits = df['profit'].tolist()
    schedule = [None] * n  # hierin komt welke job per slot
    for job in solution:
        deadline = min(deadlines[job], n)  # max n
        # zoek het laatste vrije slot vóór de deadline
        while deadline > 0 and schedule[deadline-1] is not None:
            deadline -= 1
        if deadline > 0:
            schedule[deadline-1] = job
    # tel de winst op van alle geplande jobs
    total_profit = sum(profits[j] for j in schedule if j is not None)
    return total_profit

def evaluate(population, df):
    return [fitness(sol, df) for sol in population]

def parent_selection(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    if total_fitness <= 0:
        # fallback: kies twee willekeurige ouders als gewichten ongeldig zijn
        return random.sample(population, 2)
    selection_probs = [f / total_fitness for f in fitness_scores]
    parents = random.choices(population, weights=selection_probs, k=2)
    return parents

def order_crossover(parents):
    # vervangt single_point_crossover voor permutaties (Order Crossover)
    p1, p2 = parents[0], parents[1]
    n = len(p1)
    a, b = sorted(random.sample(range(n), 2))
    child1 = [-1]*n
    child1[a:b+1] = p1[a:b+1]
    fill = (b+1) % n
    for i in range(n):
        gene = p2[(b+1+i) % n]
        if gene not in child1:
            child1[fill] = gene
            fill = (fill+1) % n
    child2 = [-1]*n
    child2[a:b+1] = p2[a:b+1]
    fill = (b+1) % n
    for i in range(n):
        gene = p1[(b+1+i) % n]
        if gene not in child2:
            child2[fill] = gene
            fill = (fill+1) % n
    return child1, child2

def mutate_swap(individual, rate):
    if random.random() < rate:
        m1, m2 = random.sample(range(len(individual)), 2)
        individual[m1], individual[m2] = individual[m2], individual[m1]
    return individual

In [ ]:
# GA parameters
N_POP = 100
N_JOBS = len(df)
N_ITERS = 2000
MUTATION_RATE = 0.1

# Maak populatie aan (lijst van n_pop willekeurige volgorde van n_jobs)
population = initialise_population(N_POP, N_JOBS)

fitness_scores = evaluate(population, df)

best_overall = None
best_score = -float('inf')  # hoger is beter (jouw fitness retourneert total_profit)
history = []

for it in range(1, N_ITERS+1):
    # update fitness
    fitness_scores = evaluate(population, df)

    # vul nieuwe populatie volledig met nakomelingen (geen elitisme)
    new_pop = []
    while len(new_pop) < N_POP:
        parents = parent_selection(population, fitness_scores)
        c1, c2 = order_crossover(parents)
        # gebruik bestaande mutate_swap functie (geeft permutatie terug)
        new_pop.append(mutate_swap(c1, MUTATION_RATE))
        if len(new_pop) < N_POP:
            new_pop.append(mutate_swap(c2, MUTATION_RATE))

    population = new_pop
    fitness_scores = evaluate(population, df)

    # update beste oplossing
    idx_best = max(range(len(population)), key=lambda i: fitness_scores[i])
    if fitness_scores[idx_best] > best_score:
        best_score = fitness_scores[idx_best]
        best_overall = population[idx_best].copy()
        

    history.append(best_score)

    if it == 1 or it % 10 == 0:
        print(f"Iter {it}/{N_ITERS} best profit: {best_score}")

# Resultaat tonen
print("\nBeste gevonden profit:", best_score)
# best_overall is lijst van DataFrame-rijnummers (0-based in de gesorteerde df)
print("Best volgorde (DataFrame indices):", best_overall)

# map naar originele job ids en toon details
mapped = [(idx, int(df.loc[idx,'job_id']), float(df.loc[idx,'profit']), int(df.loc[idx,'deadline'])) for idx in best_overall]
print("Best volgorde met job_id, profit, deadline (in order):")
for pos, job_id, profit, deadline in mapped:
    print(f"pos={pos} -> job_id={job_id}, profit={profit}, deadline={deadline}")

Iter 1/2000 best profit: 726
Iter 10/2000 best profit: 802
Iter 20/2000 best profit: 802
Iter 30/2000 best profit: 841
Iter 40/2000 best profit: 841
Iter 50/2000 best profit: 841
Iter 60/2000 best profit: 841
Iter 70/2000 best profit: 841
Iter 80/2000 best profit: 841
Iter 90/2000 best profit: 841
Iter 100/2000 best profit: 841
Iter 110/2000 best profit: 841
Iter 120/2000 best profit: 841
Iter 130/2000 best profit: 841
Iter 140/2000 best profit: 841
Iter 150/2000 best profit: 841
Iter 160/2000 best profit: 841
Iter 170/2000 best profit: 841
Iter 180/2000 best profit: 841
Iter 190/2000 best profit: 841
Iter 200/2000 best profit: 841
Iter 210/2000 best profit: 841
Iter 220/2000 best profit: 841
Iter 230/2000 best profit: 841
Iter 240/2000 best profit: 841
Iter 250/2000 best profit: 841
Iter 260/2000 best profit: 841
Iter 270/2000 best profit: 841
Iter 280/2000 best profit: 841
Iter 290/2000 best profit: 841
Iter 300/2000 best profit: 841
Iter 310/2000 best profit: 841
Iter 320/2000 best 